In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import wiener, savgol_filter

from inference import raw, fir_filter, iir_filter

%matplotlib qt

## Load raw ECG

In [ ]:
signal_path: str = './Delineation_Datasets/AttCHR0102rcs4_Sinius/13-20-54.EDF'
ecg_raw, fs_raw = raw.load_data(signal_path)

## Load processed ECG

In [ ]:
ecg_data = pd.read_csv('./Delineation_Datasets/AttCHR0102rcs4_Sinius/13-20-54_data.csv')
ecg_proc = ecg_data['signal'].to_numpy() * 1_000

## Wiener

In [ ]:
ecg_wiener_10 = wiener(ecg_raw, mysize=10)
ecg_wiener_5 = wiener(ecg_raw, mysize=5)

## Savgol

In [ ]:
savgol_10_3 = savgol_filter(ecg_raw, window_length=10, polyorder=3)
savgol_5_3 = savgol_filter(ecg_raw, window_length=5, polyorder=3)

## FIR 

In [ ]:
cutoff_low = 0.67
cutoff_high = 100.0
factor = 4.5

fir, N = fir_filter.FIRRemoveBL(ecg_raw, fs_raw, cutoff_low, factor)
fir, N = fir_filter.FIRRemoveHF(fir, fs_raw, cutoff_high, factor)

## IIR

In [ ]:
cutoff_low = 0.67
cutoff_high = 100.0

iir = iir_filter.IIRRemoveBL(ecg_raw, fs_raw, cutoff_low)
iir = iir_filter.IIRRemoveHF(iir, fs_raw, cutoff_high)

## Plots

In [ ]:
start = 0
end = 10_000

fig, ax = plt.subplots()
ax.set_title('Click on legend line to toggle line on/off')

line1, = ax.plot(ecg_raw[start:end], label='raw')

line2, = ax.plot(ecg_proc[start:end], label='ours')

line3, = ax.plot(ecg_wiener_10[start:end], label='wiener_10')
line4, = ax.plot(ecg_wiener_5[start:end], label='wiener_5')

line5, = ax.plot(savgol_10_3[start:end], label='savgol_10_3')
line6, = ax.plot(savgol_5_3[start:end], label='savgol_5_3')

line7, = ax.plot(fir[start:end], label='fir')

line8, = ax.plot(iir[start:end], label='iir')

plt.axhline(y=0, lw=0.6, ls="--")
leg = ax.legend(fancybox=True, shadow=True)

lines = [line1, line2, line3, line4, line5, line6, line7, line8]
lined = {}  # Will map legend lines to original lines.
for legline, origline in zip(leg.get_lines(), lines):
    legline.set_picker(True)  # Enable picking on the legend line.
    lined[legline] = origline


def on_pick(event):
    # On the pick event, find the original line corresponding to the legend
    # proxy line, and toggle its visibility.
    legline = event.artist
    origline = lined[legline]
    visible = not origline.get_visible()
    origline.set_visible(visible)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled.
    legline.set_alpha(1.0 if visible else 0.2)
    fig.canvas.draw()

fig.canvas.mpl_connect('pick_event', on_pick)
plt.show()